# import dependencies

In [167]:
import os
import sys
import pandas as pd
import re
import pickle

# Obtener la ruta del directorio padre
parent_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Añadir la ruta del directorio padre al path de Python
sys.path.append(parent_path)
from connect import *
from dataStructure import *

# Paths to raw data

In [143]:
raw_data_folder_path = os.path.join(parent_path,'rawData')

# Loading and cleaning data

In [144]:
raw_data_name_files = os.listdir(raw_data_folder_path)

In [145]:
file_to_read = 'TES3Z'
file = [name_file for name_file in raw_data_name_files if file_to_read in name_file][0]
path_file = os.path.join(raw_data_folder_path,file)
df = pd.read_excel(path_file, index_col=0)

In [146]:
tes = {
    'tesName': [],
    'tags':[],
    'stackw':[],
    'stackl':[],
    'stackMoT':[],
    'stackAu1T':[],
    'stackAu2T':[],
    'absorber':[],
    'membrane':[],
    'TC0':[],
    'RN':[]
}

def verifica_patron_NZN(cadena):
    '''
    patron a verificar número Z número
    '''
    patron = r"\d[Z]\d+$"
    return bool(re.match(patron, cadena))

def verifica_patron_2d(cadena):
    '''
    Verifica dos dígitos
    '''
    patron = r"^\d{2}$"
    return bool(re.match(patron, cadena))

def verifica_patron_3numeros(cadena):
    '''
    Verifica que la cadena de texto contenga como máximo tres números
    '''
    patron = '^\D*(\d\D*){1,3}$'
    return bool(re.match(patron, cadena))



def clean_names(name:str, i:int)->str:
    '''
    clean strings names
    '''
    name_list = name.split('_')

    if len(name_list) != 3:
        print(f'The name {name} needs further inspection, position {i}')
        final_name = name
        return final_name

    if verifica_patron_NZN(name_list[0]) == True:
        final_name =name_list[0]+'_'
    else:
        print(f'The name {name} needs further inspection, position {i}')
        final_name = name
        return final_name
    
    if verifica_patron_2d(name_list[1].strip()) == True:
        final_name = final_name+name_list[1].strip()+'_'
    else:
        print(f'The name {name} needs further inspection, position {i}')
        final_name = name
        return final_name
    
    if verifica_patron_2d(name_list[2].strip()[:2]) == True or i == 8:
        if verifica_patron_2d(name_list[2].strip()[:2]) == True:
            final_name = final_name+name_list[2].strip()[:2]
        else:
            final_name = final_name+name_list[2].strip()
    else:
        print(f'The name {name} needs further inspection, position {i}')
        final_name = name
        return final_name
    
    return final_name


def clean_width_length(raw:str, i:int)->List:

    data = raw.lower().split('x')

    try:
        data = [int(i.strip()) for i in data]
    except Exception as error:
        data = [-1,-1]
        message = f"An error occurred: {error} and the data {i} was not properly transformed"
        print(message)
    
    return data

def clean_Mo_Au_Au(raw:str, i:int)->List:
    final_data = []
    data = raw.lower().split('/')

    if len(data)!=3:
        final_data  = data
        print(f'Verifica dato en posición {i}')
        return final_data
        
    for d in data:
        if verifica_patron_3numeros(d.strip()) == True:
            final_data.append(int(d.strip()))
        else:
            print(f'Verifica dato en posición {i}')
            return final_data
    return final_data
    
tes['tesName'] = [clean_names(name, i) for i,name in enumerate(df.columns)]

tes['tags'] = list(zip(*[[row[name] for name in df.columns] for n, row in df.iterrows() if n == 'Tags' or n == 'Tags_']))

wl = list(zip(*[[clean_width_length(data,i) for i, data in enumerate(row)] for n, row in df.iterrows() if n == 'stack Area WxL (um)'][0]))

tes['stackw'] = list(wl[0])

tes['stackl'] = list(wl[1])

mo1o2 = list(zip(*[[clean_Mo_Au_Au(data,i) for i, data in enumerate(row)] for n, row in df.iterrows() if n == 'stackThickMoAuAu (nm)'][0]))

tes['stackMoT'] = list(mo1o2[0])

tes['stackAu1T'] = list(mo1o2[1])

tes['stackAu2T'] = list(mo1o2[2])

tes['absorber'] = [[data.lower().strip() for data in row] for n, row in df.iterrows() if n == 'Absorbente'][0]

tes['membrane'] = [[data.lower().strip() for data in row] for n, row in df.iterrows() if n == 'Membrana'][0]

tes['TC0'] = [[int(data) for data in row] for n, row in df.iterrows() if n == 'Tc0 mK'][0]

tes['RN'] = [[float(data) for data in row] for n, row in df.iterrows() if n == 'RN Ohms'][0]

df_raw = pd.DataFrame.from_dict(tes)


An error occurred: invalid literal for int() with base 10: 'no' and the data 8 was not properly transformed


## Further cleaning

In [150]:
duplicated_names = df_raw['tesName'][df_raw['tesName'].duplicated()].to_list()
for d_name in duplicated_names:
    indice = []
    state = []
    for n,row in df_raw[df_raw['tesName']==d_name].iterrows():
        state.append(row['tags'][1])
        indice.append(n)
    if all(x == state[0] for x in state)==False:
        to_delete = state.index('Funcional')
        df_raw = df_raw.drop(indice[to_delete])

In [163]:
column_names_to_aggregate = list(df_raw.columns)
column_names_to_aggregate.pop(column_names_to_aggregate.index('TC0'))
df_agg = df_raw.groupby(column_names_to_aggregate)['TC0'].apply(lambda x: list(x)).reset_index(name='TC0')
df_agg.head()

,tesName,tags,stackw,stackl,stackMoT,stackAu1T,stackAu2T,absorber,membrane,RN,TC0
0,3Z10_33_34,"(EB, Funcional)",30,16,40,15,200,no,no,16.0,[92]
1,3Z10_53_13,"(EB, Funcional)",16,10,40,15,200,no,no,19.0,[100]
2,3Z10_53_33,"(EB, Funcional)",10,16,40,15,200,no,no,48.0,[85]
3,3Z10_64_14,"(EB, Funcional)",16,8,40,15,200,no,no,15.0,[170]
4,3Z10_64_41,"(EB, Funcional)",16,24,40,15,200,no,no,43.0,[90]


In [165]:
df_agg['tesName'][df_agg['tesName'].duplicated()]

Series([], Name: tesName, dtype: object)

# Saving data

In [169]:
store_path = os.path.join(parent_path,'OutputData')
file_path_out = os.path.join(store_path,'df_agg.pickle')
with open(file_path_out, 'wb') as handle:
    pickle.dump(df_agg,handle)